# Advanced Dataframes

## Part 3 - Reshaping
- crosstab
- pivot_table

In [15]:
#standard imports
import pandas as pd
import numpy as np

np.random.seed(123)

In [16]:
# Create list of values for names column.
students = ['Sally', 'Jane', 'Suzie', 'Billy', 'Ada', 'John', 'Thomas',
            'Marie', 'Albert', 'Richard', 'Isaac', 'Alan']

# Randomly generate arrays of scores for each student for each subject.
# Note that all the values need to have the same length here.
math_grades = np.random.randint(low=60, high=100, size=len(students))
english_grades = np.random.randint(low=60, high=100, size=len(students))
reading_grades = np.random.randint(low=60, high=100, size=len(students))

# Randomly generate if a student is in classroom A or classroom B
classroom = np.random.choice(['A', 'B'], len(students))

df = pd.DataFrame({'name':students,
                   'math':math_grades,
                   'reading':reading_grades,
                   'english':english_grades,
                   'room':classroom})

#calculated overall grade and letter
df['overall_grade'] = df[['math','reading','english']].mean(axis=1).round(1)

#np.where(condition, output if true, output if false)
df['overall_letter'] = np.where(df.overall_grade >= 90, 'A', 
                                np.where(df.overall_grade >= 80, 'B','C'))

df

,name,math,reading,english,room,overall_grade,overall_letter
0,Sally,62,80,85,A,75.7,C
1,Jane,88,67,79,B,78.0,C
2,Suzie,94,95,74,A,87.7,B
3,Billy,98,88,96,B,94.0,A
4,Ada,77,98,92,A,89.0,B
5,John,79,93,76,B,82.7,B
6,Thomas,82,81,64,A,75.7,C
7,Marie,93,90,63,A,82.0,B
8,Albert,92,87,62,A,80.3,B
9,Richard,69,94,80,A,81.0,B


## Using `.crosstab`
count the number of occurances between each subgroup
- format: `pd.crosstab(index=_, column=_)`

#### find all counts of the room and overall letter grade

In [18]:
pd.crosstab(df.room, df.overall_letter)

overall_letter,A,B,C
room,,,
A,0,5,3
B,2,1,1


#### add total counts

In [19]:
#margins
pd.crosstab(df.room, df.overall_letter, margins=True)

overall_letter,A,B,C,All
room,,,,
A,0,5,3,8
B,2,1,1,4
All,2,6,4,12


#### give percentages instead of counts

In [22]:
pd.crosstab(df.room, df.overall_letter)

overall_letter,A,B,C
room,,,
A,0,5,3
B,2,1,1


In [21]:
#normalize
pd.crosstab(df.room, df.overall_letter, normalize=True)

overall_letter,A,B,C
room,,,
A,0.000000,0.416667,0.250000
B,0.166667,0.083333,0.083333


In [30]:
#normalize
pd.crosstab(df.room, df.overall_letter, normalize='index')

overall_letter,A,B,C
room,,,
A,0.0,0.625,0.375
B,0.5,0.250,0.250


In [31]:
#normalize
pd.crosstab(df.room, df.overall_letter, normalize='columns')

overall_letter,A,B,C
room,,,
A,0.0,0.833333,0.75
B,1.0,0.166667,0.25


### Using `.pivot_table`

creates a summary table similar to excel pivot table 

need to include 3 of the following:
- which values will make up the rows (the index)
- which values will make up the columns
- the values we are aggregating
- an aggregation method (aggfunc); if we can omit this, and mean will be used by default

format: `pd.pivot_table(data=_, index=_, columns=_, values=_, aggfunc=_)`

#### show the average math grade by overall letter grade and room

In [34]:
pd.pivot_table(data=df, index='room', columns='overall_letter', 
               values='math')

overall_letter,A,B,C
room,,,
A,NaN,85.0,78.666667
B,95.0,79.0,88.000000


In [42]:
#all people in room A with an overall letter grade of B
df [(df.room == 'A') & (df.overall_letter == 'B')]

,name,math,reading,english,room,overall_grade,overall_letter
2,Suzie,94,95,74,A,87.7,B
4,Ada,77,98,92,A,89.0,B
7,Marie,93,90,63,A,82.0,B
8,Albert,92,87,62,A,80.3,B
9,Richard,69,94,80,A,81.0,B


#### show the MAX math grade by overall letter grade and room

In [44]:
pd.pivot_table(data=df, index='room', columns='overall_letter',
              values='math', aggfunc='max')

overall_letter,A,B,C
room,,,
A,NaN,94.0,92.0
B,98.0,79.0,88.0


> `index` and `columns` should be categorical values, and `values` should be numeric

In [46]:
#not what we want
pd.pivot_table(data=df, index='room', columns='overall_grade',
              values='math', aggfunc='max')

overall_grade,75.3,75.7,78.0,80.3,81.0,82.0,82.7,87.7,89.0,94.0,94.7
room,,,,,,,,,,,
A,92.0,82.0,NaN,92.0,69.0,93.0,NaN,94.0,77.0,NaN,NaN
B,NaN,NaN,88.0,NaN,NaN,NaN,79.0,NaN,NaN,98.0,92.0


### Using `.map`

Use a dictionary to assign new values based on current values

- format: `.map({'old_value':'new_value'})`

#### create new df of restuarant data

In [47]:
n = 40

df = pd.DataFrame({
    'drink': np.random.choice(['Tea', 'Water', 'Water'], n),
    
    'meal': np.random.choice(['Curry', 'Yakisoba Noodle', 'Pad Thai'], n),
})

df.sample(10)

,drink,meal
0,Tea,Pad Thai
38,Water,Curry
11,Tea,Curry
4,Tea,Curry
16,Water,Curry
2,Tea,Yakisoba Noodle
22,Water,Yakisoba Noodle
5,Water,Pad Thai
19,Tea,Pad Thai
15,Water,Yakisoba Noodle


#### what are all unique drink values

In [49]:
df.drink.unique()

array(['Tea', 'Water'], dtype=object)

#### what are all unique meal values

In [50]:
df.meal.unique()

array(['Pad Thai', 'Yakisoba Noodle', 'Curry'], dtype=object)

#### create dictionary of prices for all current items

In [52]:
prices = {
    'Yakisoba Noodle': 9,
    'Curry': 11,
    'Pad Thai': 10,
    'Tea': 2,
    'Water': 0,
}

#### map drink prices and meal prices to create bill

In [57]:
df.drink.head()

0      Tea
1    Water
2      Tea
3    Water
4      Tea
Name: drink, dtype: object

In [58]:
df.drink.map(prices).head()

0    2
1    0
2    2
3    0
4    2
Name: drink, dtype: int64

In [60]:
df.meal.map(prices).head()

0    10
1    10
2     9
3    10
4    11
Name: meal, dtype: int64

In [69]:
df['bill'] = df.drink.map(prices)  + df.meal.map(prices)

In [70]:
df

,drink,meal,bill
0,Tea,Pad Thai,12
1,Water,Pad Thai,10
2,Tea,Yakisoba Noodle,11
3,Water,Pad Thai,10
4,Tea,Curry,13
5,Water,Pad Thai,10
6,Water,Pad Thai,10
7,Water,Yakisoba Noodle,9
8,Tea,Yakisoba Noodle,11
9,Water,Pad Thai,10


#### how many orders of each combination of meal and drink are there?

In [71]:
pd.crosstab(df.meal, df.drink)

drink,Tea,Water
meal,,
Curry,3,9
Pad Thai,4,10
Yakisoba Noodle,5,9


In [72]:
pd.crosstab(df.drink, df.meal)

meal,Curry,Pad Thai,Yakisoba Noodle
drink,,,
Tea,3,4,5
Water,9,10,9


#### what percentage of each?

In [78]:
pd.crosstab(df.meal, df.drink, normalize=True) * 100

drink,Tea,Water
meal,,
Curry,7.5,22.5
Pad Thai,10.0,25.0
Yakisoba Noodle,12.5,22.5


#### whats the average bill for each combination?

In [79]:
pd.pivot_table(df, 'bill', 'meal', 'drink')

drink,Tea,Water
meal,,
Curry,13,11
Pad Thai,12,10
Yakisoba Noodle,11,9


#### we can also get this by using a groupby

In [80]:
df.groupby(['meal','drink']).mean()

bill
meal            drink      
Curry           Tea    13.0
                Water  11.0
Pad Thai        Tea    12.0
                Water  10.0
Yakisoba Noodle Tea    11.0
                Water   9.0